*주택 가격 예측하기*<br/>



설명 변수 [79개] <br/>
이 데이터를 바탕으로 모델 작성 후에 1459채 가격 예측<br/>
연속적인 값을 예측하는 회귀분석을 사용함.<br/>
RMSE = Root Mean Squared Error<br/>
RMSE를 사용할 때는 예측값과 실제 값 차이가 너무 큰 데이터가 포함되어 있으면 다른 예측값의 정확도가 아무리 좋아도 전체 예측 정확도가 크게 영향을 받음. <br/>
반면 실제 값 예측 값 각각에 로그를 취하면 각 데이터 간 차이가 작아져 특정 데이터가 전체 평가에 영향을 미치는 정도를 줄일 . 수있음.
예측 정확도를 확인할 때는 로그를 취해야 한다는 것을 잊지 맙시다.<br/>
